In [2]:
import medmnist
from medmnist import INFO

import torch
import torchvision.transforms as transforms
import torch.utils.data as data
import numpy as np

# chestmnist, retinamnist
def get_image_mean_std(dataname):
    info = INFO[dataname]
    DataClass = getattr(medmnist, info['python_class'])

    transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Resize((224, 224))
            ])

    train_dataset = DataClass(split='train', transform=transform, download=True)

    train_loader = data.DataLoader(dataset=train_dataset, batch_size=8192)
    total = info['n_samples']['train']
    mean = torch.zeros(info['n_channels'])
    std = torch.zeros(info['n_channels'])
    for images, _ in train_loader:
        num_img = len(images)
        m, s = images.mean([0,2,3]), images.std([0,2,3])
        mean += num_img * m / total
        std += np.sqrt(num_img/total) * s
    return mean, std

In [19]:
mean_stds = {}

for k in ['pathmnist', 'chestmnist', 'dermamnist', 'octmnist', 'pneumoniamnist', 'retinamnist', 'breastmnist', 'bloodmnist', 'tissuemnist', 'organamnist', 'organcmnist', 'organsmnist']:
    mean, std = get_image_mean_std(k)
    print(k, mean, std)
    mean_stds[k] = {
        'mean': mean,
        'std': std
    }
print(mean_stds)

Using downloaded and verified file: /Users/naomileow/.medmnist/pathmnist.npz
pathmnist tensor([0.7405, 0.5330, 0.7058]) tensor([0.3920, 0.5636, 0.3959])


  0%|          | 0/82802576 [00:00<?, ?it/s]

chestmnist tensor([0.4936]) tensor([0.7392])


  0%|          | 0/19725078 [00:00<?, ?it/s]

dermamnist tensor([0.7631, 0.5381, 0.5614]) tensor([0.1354, 0.1530, 0.1679])


...

  0%|          | 0/15527535 [00:00<?, ?it/s]

organcmnist tensor([0.4932]) tensor([0.3762])


  0%|          | 0/16528536 [00:00<?, ?it/s]

organsmnist tensor([0.4950]) tensor([0.3779])


  0%|          | 0/32657407 [00:00<?, ?it/s]

In [1]:
from utils.device import get_device
from models.backbone.datasets import MEAN_STDS, DataSets
from models.backbone.trainer import Trainer

device = get_device()
MODEL_SAVE_PATH = 'models/backbone/pretrained'

In [3]:
import torchvision
import torch
from torchvision.models import ResNet50_Weights
from torch import nn

retina_ds = DataSets('retinamnist', MEAN_STDS) # 5 classes
backbone = torchvision.models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
backbone.fc = nn.Linear(2048, len(retina_ds.info['label']))

batch_size = 256
rtrainer = Trainer(backbone, retina_ds, batch_size, device, balance=False)

Using downloaded and verified file: /Users/naomileow/.medmnist/retinamnist.npz
Using downloaded and verified file: /Users/naomileow/.medmnist/retinamnist.npz
Using downloaded and verified file: /Users/naomileow/.medmnist/retinamnist.npz


In [ ]:
rtrainer.run_train(5)

In [20]:
print(rtrainer.run_eval(rtrainer.best_model, rtrainer.test_loader)) 

(0.52, 0.7557038049792416, 1.3845270824432374)


In [ ]:
import os

# (0.5125, 0.7247683647010932, 1.464751205444336) for non pretrained, non balanced
print(rtrainer.run_eval(rtrainer.best_model, rtrainer.test_loader)) 

torch.save(rtrainer.best_model.state_dict(), os.path.join(MODEL_SAVE_PATH, 'retina_backbone_pretrained_bal.pkl'))

In [10]:
import torchvision

# 'chestmnist', 'pneumoniamnist', 'octmnist',  'retinamnist'
chest_ds = DataSets('chestmnist', MEAN_STDS)

backbone = torchvision.models.resnet50(num_classes=len(chest_ds.info['label']), pretrained=False)
# patch for single channel
backbone.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

batch_size = 128
ctrainer = Trainer(backbone, chest_ds, batch_size, device)

ctrainer.run_train(30)

Using downloaded and verified file: /Users/naomileow/.medmnist/chestmnist.npz
Using downloaded and verified file: /Users/naomileow/.medmnist/chestmnist.npz
Using downloaded and verified file: /Users/naomileow/.medmnist/chestmnist.npz


/opt/homebrew/anaconda3/envs/cs6240/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/anaconda3/envs/cs6240/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [10]:
import torchvision
import os
import torch

# [7996,  1950,  9261, 13914,  3988,  4375,   978,  3705,  3263, 1690,  1799,  1158,  2279,   144]
chest_ds = DataSets('chestmnist', mean_stds)

backbone = torchvision.models.resnet50(num_classes=len(chest_ds.info['label']), weights=None)
backbone.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
backbone.load_state_dict(torch.load(os.path.join(MODEL_SAVE_PATH, 'cxr_backbone.pkl')))

batch_size = 256
ctrainer = Trainer(backbone, chest_ds, batch_size, device, balance=True)

Using downloaded and verified file: /Users/naomileow/.medmnist/chestmnist.npz
Using downloaded and verified file: /Users/naomileow/.medmnist/chestmnist.npz
Using downloaded and verified file: /Users/naomileow/.medmnist/chestmnist.npz


/opt/homebrew/anaconda3/envs/cs6240/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/anaconda3/envs/cs6240/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
ctrainer.run_train(5)

In [14]:
ctrainer.run_eval(ctrainer.model, ctrainer.test_loader)

(0.9239799784755875, 0.6329840270919405, 0.7261211995067128)

In [ ]:
# Load and save pretrained resnet from medclip
import torch
from torch import nn

from medclip import MedCLIPModel, MedCLIPVisionModel

# load MedCLIP-ResNet50
model = MedCLIPModel(vision_cls=MedCLIPVisionModel)
model.from_pretrained()

bconv_weight = model.vision_model.model.conv1.weight.mean(dim=1).unsqueeze(1)

# The resnet model was trained on CheXpert and MIMIC-CXR
backbone = model.vision_model.model
backbone.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
backbone.conv1.weight = nn.Parameter(bconv_weight)

torch.save(backbone.state_dict(), 'medclip_resnet50.pkl')

In [1]:
import torch
import pandas as pd
from torch.utils.data import DataLoader

from utils.labels import VINDR_CXR_LABELS, VINDR_SPLIT, VINDR_SPLIT2
from models.backbone.datasets import MEAN_STDS

from utils.data import get_query_and_support_ids, DatasetConfig
from utils.device import get_device
from models.embedding.dataset import Dataset
from utils.sampling import MultilabelBalancedRandomSampler

configs = {
    'vindr2': DatasetConfig('datasets/vindr-cxr-png', 'data/vindr_cxr_split_labels2.pkl', 'data/vindr_train_query_set2.pkl', VINDR_CXR_LABELS, VINDR_SPLIT2, MEAN_STDS['chestmnist'])
}

config = configs['vindr2']

batch_size = 10*10
query_image_ids, support_image_ids = get_query_and_support_ids(config.img_info, config.training_split_path)
query_dataset = Dataset(config.img_path, config.img_info, query_image_ids, config.label_names_map, config.classes_split_map['train'], mean_std=config.mean_std)
query_loader = DataLoader(dataset=query_dataset, batch_size=batch_size, shuffle=True)
support_dataset = Dataset(config.img_path, config.img_info, support_image_ids, config.label_names_map, config.classes_split_map['train'], mean_std=config.mean_std)
support_loader = DataLoader(dataset=support_dataset, batch_size=batch_size, sampler=MultilabelBalancedRandomSampler(support_dataset.get_class_indicators()))

PROJ_SIZE = 512
device = get_device()

In [2]:
import torchvision
import os
import torch
from models.backbone.trainer import DSTrainer

backbone = torchvision.models.resnet50(num_classes=len(config.classes_split_map['train']), weights=None)
backbone.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

mtrainer = DSTrainer(backbone, query_dataset.class_labels(), device=device)

In [3]:
mtrainer.run_train(10, support_loader, query_loader, lr=1e-4)

Batch 1: loss 0.7594740390777588
Batch 2: loss 0.7567426562309265
Batch 3: loss 0.7553663651148478
Batch 4: loss 0.7501944750547409
Batch 5: loss 0.7435857892036438
Batch 6: loss 0.7360113759835561
Batch 7: loss 0.7286982706614903
Batch 8: loss 0.7190685793757439
Batch 9: loss 0.7093484136793349
Batch 10: loss 0.7008466958999634
Batch 11: loss 0.6924483288418163
Batch 12: loss 0.6842299898465475
Batch 13: loss 0.6778851472414457
Batch 14: loss 0.6741286473614829
Batch 15: loss 0.6680373112360637
Batch 16: loss 0.6633482836186886
Batch 17: loss 0.6590615791432998
Batch 18: loss 0.6545084714889526
Batch 19: loss 0.6504338791495875
Batch 20: loss 0.6483506679534912
Batch 21: loss 0.6452367703119913
Batch 22: loss 0.6427820920944214
Batch 23: loss 0.6403462601744611
Batch 24: loss 0.6381940320134163
Batch 25: loss 0.6357972574234009
Batch 26: loss 0.633175634420835
Batch 27: loss 0.6309664536405493
Batch 28: loss 0.6286095295633588
Batch 29: loss 0.6263313272903706
Batch 30: loss 0.6244158

In [6]:
torch.save(mtrainer.best_model.state_dict(), 'models/backbone/pretrained/vindr2/trained-backbone-weights.pkl')